In [17]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int

In [18]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


# Existing Quantization schemes

In [19]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")


class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    
#%% testing qsgd quantization

print("QSGD")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = QSGD(2,True)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")



QSGD
Before Quantization:  [4.0170817  5.5414443  4.5256143  9.096871   1.9471809  9.134307
 1.8100957  3.7860327  4.746945   8.966664   2.4190016  8.619185
 9.590424   9.279589   2.6449683  0.86237276]
Encoded b'&\x00\x00\x00\x9f\xcf\xc7A2\x99L&\x03\x00\x00\x00'
After Quantization:  [12.488189  0.        0.       12.488189  0.        0.       12.488189
  0.        0.       12.488189  0.        0.       12.488189  0.
  0.        0.      ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [20]:
#%%SETTING UP Julia and gzip Class
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/gzip.jl")

import numpy as np

class GZip(Compressor):
    def __init__(self, s: int):
        self.s = s

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_gzip(array, self.s, seed)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_gzip(array, self.s)
        return result


#%% testing gzip quantization
print("GZip")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = GZip(1)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


GZip
Before Quantization:  [3.9092782 8.633677  0.6729474 8.55325   0.5504294 0.6945684 4.0168853
 8.292091  6.932689  2.8777552 6.18023   2.588353  4.4716864 9.178723
 3.4547925 9.145802 ]
Encoded b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\ncd\x00\x01F0\x02\x81\x85Kw;\x02\x00\xe3y\xbd\x1a\x14\x00\x00\x00'
After Quantization:  [23.455873  0.        0.        0.        0.        0.       23.455873
  0.        0.       23.455873  0.        0.        0.        0.
  0.        0.      ]
37.5% smaller 
61.30952380952381% smaller 


In [21]:
#%%FP8

# print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/fp8.jl")
# print("Finished starting up Julia.")

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)


class FP8(Compressor):
    def __init__(self):
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
#%% testing fp32 quantization
print("FP8")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = FP8()
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


FP8
Before Quantization:  [4.546948  3.220327  3.9612577 2.6818104 8.028429  7.64841   9.979168
 5.6285286 7.460853  1.0775675 6.1575527 5.4162693 1.6591051 6.0837364
 3.020323  0.8670955]
Encoded b'EBDAHGIEH=FE?FB:'
After Quantization:  [ 5.    3.    4.    2.5   8.    7.   10.    5.    8.    1.25  6.    5.
  1.75  6.    3.    0.75]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [66]:

import cv2


class Custom_linear():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    



#%%

# data = np.random.rand(1,16)

data = list(np.random.rand(1,16).flatten()*10)
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint8
cmp = customCompression(data_type)
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))


[1.7890071  4.5856886  4.6762333  1.3002442  0.6006638  8.751797
 7.3242555  2.2552674  0.19456881 6.948132   1.8482753  4.1817455
 9.428998   5.3317337  8.6860075  5.998257  ]
Encoded [ 44 121 123  30  11 236 196  56   0 186  45 110 255 141 234 160]
After Quantization:  [1.78796051 4.57639599 4.64882289 1.28097225 0.59291674 8.74094249
 7.29240458 2.22252189 0.19456881 6.9302701  1.82417396 4.17804807
 9.42899799 5.30066495 8.66851559 5.98872046]
<class 'numpy.uint8'> 48.275862068965516% smaller 
<class 'numpy.uint8'> 28.571428571428573% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  0.00035940889569238286


In [23]:
print(sys.getsizeof(data))
print(sys.getsizeof(dec))
print(sys.getsizeof(enc))

168
232
120


# Custom Frequency Domain Quantization

In [65]:
from scipy.fftpack import dct, idct

class customFreq_1_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.18340763 0.35940474 0.637309   0.5260854  0.7885159  0.14827111
 0.26648796 0.47498542 0.39220312 0.06135407 0.90992475 0.13232909
 0.29888517 0.18667614 0.5055714  0.44817612]
Encoded [ 0.3167   0.3032   0.854    0.596    0.9      0.3123   0.2363   0.4612
  0.4543   0.01152  0.7666  -0.02896  0.07434  0.1298   0.2986   0.10345]
After Quantization:  [0.18351936 0.3594447  0.6372688  0.52604955 0.7884633  0.14831954
 0.26648143 0.475038   0.39217395 0.06134844 0.91007924 0.1323649
 0.29893336 0.18667232 0.5056404  0.4481853 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  3.6322032e-09


In [25]:
from scipy.fftpack import dct, idct

class customFreq_singleDCT():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_singleDCT()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.5436643  0.675553   0.51724404 0.978212   0.64139867 0.09779149
 0.41404673 0.08269121 0.2210057  0.68379724 0.47579646 0.9324859
 0.71888214 0.12128957 0.78176266 0.49936518]
Encoded [ 2.096    0.01322  0.3833   0.2734  -0.4468  -0.3586   0.1459  -0.1213
  0.2126   0.1489   0.06128  0.1891  -0.699    0.04413 -0.04324  0.01598]
After Quantization:  [0.5435893  0.67549145 0.5170026  0.9780621  0.6412613  0.0975451
 0.41398048 0.08257373 0.22082292 0.68371403 0.47565246 0.9324037
 0.71878004 0.1210773  0.7816498  0.49920648]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  2.1824027e-08


# Custom Freq 3 (CustomFreq + Custom2 but more sophisticated)

In [26]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)
        return enc
    
    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_3():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [np.uint32, np.uint16, np.uint8]):
        self.precision_levels = precision_levels
        self.linearQuant_0 = Linear_quant(self.precision_levels[0])
        self.linearQuant_1 = Linear_quant(self.precision_levels[1])
        self.linearQuant_2 = Linear_quant(self.precision_levels[2])
        
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        self.mean_importance = np.mean(importance)
        # if self.mean_importance > 0.1:
        enc = self.linearQuant_0.encode(weights_f)  # 8-bit
        precision = int(str(self.linearQuant_0.quantization_type).split('t')[-1].split('\'')[0])
        print(precision)

        # elif self.mean_importance > 0.01:
        #     enc = self.linearQuant_1.encode(weights_f)   # 4-bit
        #     precision = int(str(self.linearQuant_1.quantization_type).split('t')[-1].split('\'')[0])
        #     # enc = self.quantize(weights_f, 16)
        # else:
        #     enc = self.linearQuant_2.encode(weights_f)   # 2-bit
        #     precision = int(str(self.linearQuant_2.quantization_type).split('t')[-1].split('\'')[0])

        
        # Quantize frequency components
        # enc = self.quantize(enc, precision)
        return enc


    def decode(self, enc) :

        # if self.mean_importance > 0.1:
        weights_quantized = self.linearQuant_0.decode(enc)  # 8-bit
        # elif self.mean_importance > 0.01:
        #     weights_quantized = self.linearQuant_1.decode(enc)   # 4-bit
        # else:
        #     weights_quantized = self.linearQuant_2.decode(enc)   # 2-bit
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_3()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.5095341  0.52359766 0.91601497 0.9974388  0.54131114 0.23511642
 0.34478948 0.78257656 0.88095915 0.77054775 0.8467786  0.5000688
 0.06885096 0.26212695 0.41739336 0.37636238]
32
Encoded [2394962614 2868525800 3571808096 4294967295 3041672279 1844622819
 2030306043 3175803985 3299889982 2391510987 2082449962  911691949
          0 1042914862 1357725588  967303079]
After Quantization:  [0.50953409 0.52359761 0.91601496 0.99743881 0.54131098 0.23511646
 0.34478939 0.78257659 0.88095908 0.77054766 0.84677845 0.50006884
 0.06885106 0.26212693 0.41739334 0.37636234]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  5.297558927501835e-15


# CustomFreq2.5

In [29]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')
    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')
    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.1183062  0.04101991 0.53759336 0.74481106 0.30189052 0.22749867
 0.9937169  0.58159703 0.68112314 0.6514682  0.10114367 0.94961864
 0.5627769  0.6438472  0.14933904 0.06646771]
Encoded [ 0.2654   0.2126   0.536    0.9683   0.6533   0.3186   1.024    0.7935
  0.4663   0.6147  -0.0776   0.733    0.1514   0.02922 -0.3647  -0.2062 ]
After Quantization:  [0.11836858 0.04111767 0.5376554  0.74456114 0.30181903 0.22741565
 0.99375236 0.58156115 0.68125314 0.65146166 0.10121679 0.94942003
 0.5627684  0.6438497  0.1492386  0.06635966]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.1117486e-08


# Custom Freq 2 (CustomFreq + Custom2) 
- best version

In [30]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_2()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.14058226 0.09781751 0.25751257 0.99536324 0.27378353 0.3666416
 0.21409433 0.37133664 0.6211666  0.33606318 0.84725195 0.52465045
 0.96947545 0.25636163 0.18093231 0.6516183 ]
Encoded [1298295913 2017200945 1543646663 4294967295 3080915924 1786187962
 2396157748 1964841294 3501745497 1886099490 3438025413 1854374400
 2462020906          0  415621134 1918417723]
After Quantization:  [0.1405822  0.09781751 0.25751244 0.99536305 0.27378351 0.36664158
 0.21409423 0.37133657 0.62116658 0.33606316 0.84725184 0.52465033
 0.96947537 0.25636164 0.18093222 0.65161823]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  7.632148421792345e-15


# Federated Implementation

In [44]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp=cmp
		self.cell_column_names = ['scheme_name','Epoch','Loss','Acc']
		self.cell = np.zeros((aggregate_epochs, len(self.cell_column_names)), dtype=object)

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        
        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100
		return np.array(weights),start_size,end_size



	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights



	def client_generator(self, train_x, train_y):
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()
		
		return weights, running_loss, start_size,end_size
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for dataset_x, dataset_y in clients:
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			self.cell[epoch][0]=self.cmp.__class__.__name__
			self.cell[epoch][1]=epoch
			self.cell[epoch][2]= round(running_loss/client, 3)
			self.cell[epoch][3]= round(test_acc.item(), 3)

			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return self.cell

#%%

In [67]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
epoch_time = time.time()
tic = time.time()
filename = f"saved_models_{epoch_time}"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customCompression(np.uint8)
# cmp = customFreq_2()

# cmp = FP8()
# cmp = QSGD(2,True)
cmp = Custom_linear()
m_8 = MNIST_PAQ_COMP(cmp = cmp, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
result = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", time.time()-tic)

  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_1592\379212063.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30107}: 100%|██████████| 164/164 [00:19<00:00,  8.41it/s]


Test Accuracy: 0.10314
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 2.04608}: 100%|██████████| 164/164 [00:15<00:00, 10.41it/s]


Test Accuracy: 0.44331
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 3, 'Loss': 1.87993}: 100%|██████████| 164/164 [00:17<00:00,  9.32it/s]


...

Test Accuracy: 0.83751
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.61981}: 100%|██████████| 164/164 [00:17<00:00,  9.17it/s]


Test Accuracy: 0.83931
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.61524}: 100%|██████████| 164/164 [00:17<00:00,  9.64it/s]


Test Accuracy: 0.84228
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  191.57473039627075


In [68]:
result =  pd.DataFrame(result, columns=m_8.cell_column_names)
file_path = r'E:\MS_Thesis\Pre-defense-june\Results_for_update\scheme_ACC_LOSS_sheet.csv'
display(result)

,scheme_name,Epoch,Loss,Acc
0,Custom_linear,0,2.301,0.103
1,Custom_linear,1,2.046,0.443
2,Custom_linear,2,1.88,0.604
3,Custom_linear,3,1.83,0.631
4,Custom_linear,4,1.717,0.676
5,Custom_linear,5,1.643,0.82
6,Custom_linear,6,1.629,0.832
7,Custom_linear,7,1.623,0.838
8,Custom_linear,8,1.62,0.839
9,Custom_linear,9,1.615,0.842


In [70]:

if os.path.exists(file_path):
    # Read the existing CSV file
    existing_data = pd.read_csv(file_path)
    
    # Identify the last epoch in the current run
    last_epoch = result['Epoch'].max()
    
    # Iterate over the new results and update the existing ones if necessary
    for index, row in result.iterrows():
        scheme_name = row['scheme_name']
        epoch = row['Epoch']
        new_acc = row['Acc']
        
        # Check if the scheme_name and epoch already exist in the existing data
        existing_entry = existing_data[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == epoch)]
        
        if not existing_entry.empty:
            # Check if the new accuracy at the last epoch is greater than the existing one
            if epoch == last_epoch:
                existing_last_epoch_entry = existing_data[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == last_epoch)]
                if new_acc > existing_last_epoch_entry['Acc'].values[0]:
                    # Update the existing entry
                    existing_data.loc[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == epoch), ['Loss', 'Acc']] = row[['Loss', 'Acc']]
        else:
            # Append the new row to the existing data
            # existing_data = existing_data.append(row, ignore_index=True)
            existing_data = pd.concat([existing_data, pd.DataFrame([row])], ignore_index=True)

    # Save the updated data back to the CSV file
    existing_data.to_csv(file_path, index=False)
else:
    # If the file does not exist, save the new result as a new CSV file
    result.to_csv(file_path, index=False)


In [41]:
m_8.cmp.__class__.__name__

'customFreq_2'

In [ ]:
{'Epoch': 1, 'Loss': 2.30103}: 100%|██████████| 164/164 [00:36<00:00,  4.53it/s]
Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.05589}: 100%|██████████| 164/164 [00:28<00:00,  5.68it/s]
Test Accuracy: 0.27118
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.78047}: 100%|██████████| 164/164 [00:29<00:00,  5.61it/s]
Test Accuracy: 0.67313
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.64235}: 100%|██████████| 164/164 [00:28<00:00,  5.67it/s]
Test Accuracy: 0.83035
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.59154}: 100%|██████████| 164/164 [00:33<00:00,  4.87it/s]
Test Accuracy: 0.87038
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.56933}: 100%|██████████| 164/164 [00:37<00:00,  4.33it/s]
Test Accuracy: 0.8855
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.55506}: 100%|██████████| 164/164 [00:34<00:00,  4.79it/s]
Test Accuracy: 0.89718
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.54833}: 100%|██████████| 164/164 [00:35<00:00,  4.65it/s]
Test Accuracy: 0.90833
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.54572}: 100%|██████████| 164/164 [00:36<00:00,  4.54it/s]
Test Accuracy: 0.91517
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.53683}: 100%|██████████| 164/164 [00:36<00:00,  4.55it/s]
Test Accuracy: 0.92127
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  350.4498794078827

# Uint16-custom2


{'Epoch': 1, 'Loss': 2.30095}: 100%|██████████| 164/164 [00:59<00:00,  2.74it/s]
Test Accuracy: 0.09963
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.09561}: 100%|██████████| 164/164 [00:57<00:00,  2.84it/s]
Test Accuracy: 0.36056
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.86639}: 100%|██████████| 164/164 [00:58<00:00,  2.81it/s]
Test Accuracy: 0.63837
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.77721}: 100%|██████████| 164/164 [00:56<00:00,  2.90it/s]
Test Accuracy: 0.68152
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.74531}: 100%|██████████| 164/164 [01:01<00:00,  2.68it/s]
Test Accuracy: 0.71413
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.72985}: 100%|██████████| 164/164 [00:44<00:00,  3.71it/s]
Test Accuracy: 0.73011
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.64313}: 100%|██████████| 164/164 [01:02<00:00,  2.63it/s]
Test Accuracy: 0.73162
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.54832}: 100%|██████████| 164/164 [00:42<00:00,  3.83it/s]
Test Accuracy: 0.90883
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.53205}: 100%|██████████| 164/164 [00:22<00:00,  7.37it/s]
Test Accuracy: 0.92051
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.52779}: 100%|██████████| 164/164 [00:40<00:00,  4.01it/s]
Test Accuracy: 0.92549
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784

# uint8-custom 1
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:54<00:00,  3.00it/s]
Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.13939}: 100%|██████████| 164/164 [00:44<00:00,  3.65it/s]
Test Accuracy: 0.38529
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.98048}: 100%|██████████| 164/164 [00:23<00:00,  6.95it/s]
Test Accuracy: 0.44234
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.82871}: 100%|██████████| 164/164 [00:43<00:00,  3.76it/s]
Test Accuracy: 0.64131
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.77463}: 100%|██████████| 164/164 [00:25<00:00,  6.49it/s]
Test Accuracy: 0.68517
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.75027}: 100%|██████████| 164/164 [00:24<00:00,  6.64it/s]
Test Accuracy: 0.70226
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.73521}: 100%|██████████| 164/164 [00:47<00:00,  3.49it/s]
Test Accuracy: 0.71979
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.71573}: 100%|██████████| 164/164 [00:44<00:00,  3.67it/s]
Test Accuracy: 0.73043
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.68576}: 100%|██████████| 164/164 [00:53<00:00,  3.06it/s]
Test Accuracy: 0.73663
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.64119}: 100%|██████████| 164/164 [00:21<00:00,  7.65it/s]
Test Accuracy: 0.8215
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  399.71174716949463